In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torch.nn.functional as F

#from datasets import get_mnist_dataset, get_data_loader
#from utils import *
#from models import *

import pickle as pkl
import os
import datetime as dt
import pandas as pd
import random

from generate_dataloaders import *

from tqdm import tqdm_notebook as tqdm

## Get Dataloaders

In [2]:
def get_dataloaders(train_filename,val_filename):
    path = os.getcwd()
    data_dir = path + '/data/'
    train_dataloader = pkl.load(open(data_dir + train_filename,'rb'))
    val_dataloader = pkl.load(open(data_dir + val_filename,'rb'))
    return train_dataloader,val_dataloader

In [3]:
seed = 1029
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.
torch.manual_seed(seed)
torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def _init_fn(worker_id):
    np.random.seed(int(seed))

In [4]:
path = os.getcwd()
data_dir = path + '/data/'

In [5]:
train_loader,val_loader = get_dataloaders('train_dataloader.p','val_dataloader.p')

In [7]:
centroids_dataloader = pkl.load(open(data_dir + 'centroids_dataloader.p','rb'))

In [8]:
#%conda install pytorch torchvision -c pytorch
## if torch.__version__ is not 1.3.1, run this cell then restart kernel

In [9]:
print(torch.__version__)

1.3.1


## Neural Network Class

NOTE: Data loader is defined as:
- tuple: (tokens, flagged_index, problematic)

In [10]:
class neuralNetBow(nn.Module):
    """
    BagOfWords classification model
    """
    # NOTE: we can't use linear layer until we take weighted average, otherwise it will
    # remember certain positions incorrectly (ie, 4th word has bigger weights vs 7th word)
    def __init__(self, vocab_size, emb_dim, upweight=10):
        super(neuralNetBow, self).__init__()
        self.embed = nn.Embedding(vocab_size, emb_dim, padding_idx=2)
        self.upweight = upweight
    
    def forward(self, tokens, flagged_index):
        batch_size, num_tokens = tokens.shape
        embedding = self.embed(tokens)
#         print(embedding.shape) # below assumes "batch_size x num_tokens x Emb_dim" (VERIFY)
        
        # upweight by flagged_index
#         print(type(embedding))
        embedding[torch.LongTensor(range(batch_size)),flagged_index.type(torch.LongTensor),:] *= self.upweight
        
        # average across embeddings
        embedding_ave = embedding.sum(1) / (num_tokens + self.upweight - 1)
        
        return embedding_ave

### Clustering Stuff (un-tailored)

In [11]:
class KMeansCriterion(nn.Module):
    
    def __init__(self, lmbda):
        super().__init__()
        self.lmbda = lmbda
    
    def forward(self, embeddings, centroids):
        distances = torch.sum((embeddings[:, None, :] - centroids)**2, 2)
        cluster_distances, cluster_assignments = distances.min(1)
        loss = self.lmbda * cluster_distances.sum()
        return loss, cluster_assignments

In [12]:
def centroid_init(k, d, dataloader, model, current_device):
    ## Here we ideally don't want to do randomized/zero initialization
    centroid_sums = torch.zeros(k, d).to(current_device)
    centroid_counts = torch.zeros(k).to(current_device)
    for (tokens, labels, flagged_indices) in dataloader:
        # cluster_assignments = torch.LongTensor(tokens.size(0)).random_(k)
        cluster_assignments = labels.to(current_device)
        
        model.eval()
        sentence_embed = model(tokens.to(current_device),flagged_indices.to(current_device))
    
        update_clusters(centroid_sums, centroid_counts,
                        cluster_assignments, sentence_embed.to(current_device))
    
    centroid_means = centroid_sums / centroid_counts[:, None].to(current_device)
    return centroid_means.clone()

def update_clusters(centroid_sums, centroid_counts,
                    cluster_assignments, embeddings):
    k = centroid_sums.size(0)

    centroid_sums.index_add_(0, cluster_assignments, embeddings)
    bin_counts = torch.bincount(cluster_assignments,minlength=k).type(torch.FloatTensor).to(current_device)
    centroid_counts.add_(bin_counts)
    
    #np_cluster_assignments = cluster_assignments.to('cpu')
    #np_counts = np.bincount(np_cluster_assignments.data.numpy(), minlength=k)
    #centroid_counts.add_(torch.FloatTensor(np_counts))

## Load Model Info

#### model_folder changes depending on which model we're evaluating

In [32]:
## This cell will change for each model
model_folder = 'baseline_randomized_embeddings/'
model_type = neuralNetBow
criterion = KMeansCriterion(1)

In [33]:
def load_model_info(model_folder,model_type):
    path = os.getcwd()
    model_dir = path + '/models/' + model_folder

    opts = torch.load(model_dir+'opts')
    model = model_type(opts['vocab_size'], opts['emb_dim'])
    model.load_state_dict(torch.load(model_dir+'model_dict.pt',map_location=lambda storage, loc: storage))
    centroids = torch.load(model_dir+'centroids',map_location=lambda storage, loc: storage)
    
    return model, opts, centroids

In [34]:
model, opts, centroids = load_model_info(model_folder,model_type)

In [35]:
num_gpus = torch.cuda.device_count()
if num_gpus > 0:
    current_device = 'cuda'
else:
    current_device = 'cpu'
print(current_device)

cpu


In [36]:
model = model.to(current_device)
criterion = criterion.to(current_device)

# Evaluate Model

In [37]:
def forward_pass(model, centroids, val_loader, num_examples=None):
    model.eval()
    token_list = []
    cluster_assignment_list = []
    flagged_index_list = []
    original_label = []
    for i, (tokens, labels, flagged_indices) in enumerate(val_loader):
            tokens = tokens.to(current_device)
            labels = labels.to(current_device)
            flagged_indices = flagged_indices.to(current_device)
            
            # forward pass and compute loss
            sentence_embed = model(tokens,flagged_indices)
            cluster_loss, cluster_assignments = criterion(sentence_embed, centroids)
            
            # store in list
            token_list+=tokens.tolist()
            flagged_index_list+=flagged_indices.tolist()
            cluster_assignment_list+=cluster_assignments.tolist()
            original_label+=labels.tolist()
            
    return token_list, flagged_index_list, cluster_assignment_list, original_label

In [39]:
token_list, index_list, cluster_assignment_list, original_label = evaluate_model(model, centroids, val_loader)

In [40]:
len(index_list)

106

In [41]:
sum(cluster_assignment_list)

6

In [42]:
dictionary = pkl.load(open(data_dir+'dictionary.p','rb'))

In [43]:
pd.set_option('max_colwidth',0)

In [44]:
def decode_predictions(token_list,index_list,cluster_assignment_list,dictionary,original_label):
    decoded_tokens = [' '.join(dictionary.decode_idx_seq(sent)) for sent in token_list]
    reviews = [decoded for decoded in decoded_tokens]
    flagged_words = [r.split()[i] for (r,i) in zip(reviews,index_list)]
    reviews = [review.split('<pad>')[0] for review in reviews]
    df_pred = pd.DataFrame({'review':reviews,'index':index_list,'flagged_word':flagged_words,\
                            'assignment':cluster_assignment_list,'original':original_label})
    
    pred_1 = df_pred[df_pred.assignment==1]
    pred_0 = df_pred[df_pred.assignment==0]
    
    pred_1_manual_TP = len(pred_1[pred_1.original == 1]) / pred_1.shape[0]
    pred_0_manual_TP = len(pred_0[pred_0.original == 1]) / pred_0.shape[0]
    
    if pred_1_manual_TP >= pred_0_manual_TP:
        TP_cluster = pred_1
        FP_cluster = pred_0
    else:
        TP_cluster = pred_0
        FP_cluster = pred_1
        TP_cluster.assignment =0
        FP_cluster.assignment = 1

    return TP_cluster, FP_cluster

In [45]:
TP_cluster, FP_cluster = decode_predictions(token_list,index_list,cluster_assignment_list,dictionary,original_label)

In [55]:
def performance_analysis(TP_cluster,FP_cluster):
    TP_rate = len(TP_cluster[TP_cluster.original==1]) / TP_cluster.shape[0]
    FP_rate = len(TP_cluster[TP_cluster.original==0]) / TP_cluster.shape[0]
    FN_rate = len(TP_cluster[TP_cluster.original==1]) / TP_cluster.shape[0]
    TN_rate = len(TP_cluster[TP_cluster.original==0]) / TP_cluster.shape[0]
    
    accuracy = (TP_rate + TN_rate) / (TP_rate + FP_rate + FN_rate + TN_rate)
    precision = TP_rate / (TP_rate + FP_rate)
    recall = TP_rate / (TP_rate + FN_rate)
    f1_score = (2 * precision * recall) / (precision + recall)
    
    print("TP_rate:",TP_rate)
    print("FP_rate:",FP_rate)
    print("FN_rate:",FN_rate)
    print("TN_rate:",TN_rate)
    print("\n")
    
    print("Accuracy:",accuracy)
    print("Precision:",precision)
    print("Recall:",recall)
    print("F1 score:",f1_score)

In [56]:
performance_analysis(TP_cluster,FP_cluster)

TP_rate: 0.8333333333333334
FP_rate: 0.16666666666666666
FN_rate: 0.8333333333333334
TN_rate: 0.16666666666666666


Accuracy: 0.5
Precision: 0.8333333333333334
Recall: 0.5
F1 score: 0.625


In [48]:
TP_cluster

,review,index,flagged_word,assignment,original
10,experienced and talented designer who did an awesome work for the print materials for our brand .,0,experienced,1,1
16,david is obviously an expert on amazon selling techniques and had some fantastic insights .,4,expert,1,1
66,"the final product was impressive , thoughtful and completed in a timely fashion .",4,impressive,1,0
71,very creative designer.. loved how she accepted to edit frequently and was very patient.. definetly will work with holly again,1,creative,1,1
96,to start with this company tried hard to complete the tasks with enthusiasm but after i paid them the first payment they stopped communicating and completing the set tasks .,23,communicating,1,1
97,najla responded quickly to my initial enquiry and she was excited about my brief .,10,excited,1,1


In [49]:
FP_cluster

,review,index,flagged_word,assignment,original
0,hard worker .,1,worker,0,1
1,even with the limitations of a template platform like wix ultrastjarna achieved a great result .,13,great,0,0
2,very helpful and reliable !,3,reliable,0,1
3,we had a lot of wishes and demands but without hesitation he made the effort to give us the best experience possible .,20,experience,0,0
4,fast work and very friendly !,4,friendly,0,1
...,...,...,...,...,...
101,creative elements added to the logo that really captured the feeling we were looking for .,0,creative,0,0
102,excellent job and awesome communication !,0,excellent,0,0
103,quick and great work .,2,great,0,0
104,rohitha provided me with the most amazing customer experience ever .,8,experience,0,0
